In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_complaint_columns
from lib.textract import parse_textract_datetime
from lib.explore import print_df
from lib.match import (
    ColumnsIndex, StringSimilarity, JaroWinklerSimilarity, DateSimilarity, ThresholdMatcher
)
from clean.baton_rouge_pd_cprr import clean_18
from clean.baton_rouge_csd_pprr import clean_17 as clean_csd_17, clean_19 as clean_csd_19

In [2]:
df17 = clean_csd_17()
df17

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../lib/clean.py:156: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\d\.]", "").astype("float64")


,year,employee_id,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,...,hire_month,hire_day,resign_year,resign_month,resign_day,agency,uid,pay_effective_year,dept_year,rank_year
5659,2008,456780,tate,joshua,j,5050001,police department-operation serv bureau,5005,police officer,5,...,1,21,,,,Baton Rouge CSD,0004aa9e5a7fe44d2e8746aeff1d2e97,NaN,NaN,NaN
3587,2008,387088,bailey,jana,m,5010001,police department-administration,110435,senior legal specialist,0,...,3,2,,,,Baton Rouge CSD,000d35f47a90a410d73ec0ddf6dd943d,NaN,NaN,NaN
3588,2009,387088,bailey,jana,m,5010001,police department-administration,110435,senior legal specialist,0,...,3,2,,,,Baton Rouge CSD,000d35f47a90a410d73ec0ddf6dd943d,2009.0,NaN,NaN
3589,2010,387088,bailey,jana,m,5010001,police department-administration,110435,senior legal specialist,0,...,3,2,,,,Baton Rouge CSD,000d35f47a90a410d73ec0ddf6dd943d,2010.0,NaN,NaN
3590,2011,387088,bailey,jana,m,5010001,police department-administration,110435,senior legal specialist,0,...,3,2,,,,Baton Rouge CSD,000d35f47a90a410d73ec0ddf6dd943d,2011.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,2013,383910,royal,althea,p,5050001,police department-operation serv bureau,110090,clerical specialist,0,...,5,7,,,,Baton Rouge CSD,ffc4c92c0507e19003f3ffbc8a347aba,2013.0,NaN,NaN
3468,2014,383910,royal,althea,p,5050001,police department-operation serv bureau,110090,clerical specialist,0,...,5,7,,,,Baton Rouge CSD,ffc4c92c0507e19003f3ffbc8a347aba,2014.0,NaN,NaN
7553,2015,383910,royal,althea,p,5050001,police department-operation serv bureau,110132,senior clerical specialist,0,...,5,7,,,,Baton Rouge CSD,ffc4c92c0507e19003f3ffbc8a347aba,2015.0,NaN,2015.0
8876,2016,383910,royal,althea,p,5050001,police department-operation serv bureau,110132,senior clerical specialist,0,...,5,7,,,,Baton Rouge CSD,ffc4c92c0507e19003f3ffbc8a347aba,2016.0,NaN,NaN


In [3]:
df19 = clean_csd_19()
df19

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../lib/clean.py:156: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\d\.]", "").astype("float64")
../lib/clean.py:150: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.strip().str.lower()\


,year,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,annual_salary,...,hire_month,hire_day,resign_year,resign_month,resign_day,agency,uid,pay_effective_year,rank_year,dept_year
404,2018,sawyer,laterial,d,5020,police-uniform patrol bureau,5005,police officer,active,43884.51,...,2,18,,,,Baton Rouge CSD,00470dc28b790079ea4003c6ea9d9a6d,NaN,NaN,NaN
1332,2019,sawyer,laterial,d,5020,police-uniform patrol bureau,5005,police officer,active,46039.68,...,2,18,,,,Baton Rouge CSD,00470dc28b790079ea4003c6ea9d9a6d,2019.0,NaN,NaN
684,2018,blake,brian,r,5020,police-uniform patrol bureau,5010,police sergeant,active,72848.05,...,8,9,,,,Baton Rouge CSD,007f09c47926f096be2ef8013c798c19,NaN,NaN,NaN
1591,2019,blake,brian,r,5020,police-uniform patrol bureau,5010,police sergeant,active,76664.95,...,8,9,,,,Baton Rouge CSD,007f09c47926f096be2ef8013c798c19,2019.0,NaN,NaN
678,2018,ned,larry,m,5020,police-uniform patrol bureau,5010,police sergeant,active,56126.86,...,2,4,,,,Baton Rouge CSD,00e895256b8e81fdcd52c67969411cae,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,2019,brooks,charlene,j,5040,police-operational serv bureau,5235,police fingerprint tech i,inactive,7345.06,...,2,21,2018,11,30,Baton Rouge CSD,fe48657a1e3f89951ce2dbf0112cd8e3,2019.0,NaN,NaN
75,2018,pointer,sheldon,d,5020,police-uniform patrol bureau,5005,police officer,inactive,63382.81,...,6,27,2018,11,1,Baton Rouge CSD,ff3e06c89dee3ec36c1f6da1624a0b4b,NaN,NaN,NaN
199,2018,williams,liza,s,5020,police-uniform patrol bureau,5005,police officer,active,52178.13,...,11,29,,,,Baton Rouge CSD,ff858ea814c23620dcc3cfef85dded8f,NaN,NaN,NaN
1147,2019,williams,liza,s,5020,police-uniform patrol bureau,5005,police officer,active,52728.62,...,11,29,,,,Baton Rouge CSD,ff858ea814c23620dcc3cfef85dded8f,2019.0,NaN,NaN


In [6]:
df19[df19.dept_year.notna()]
df19[df19.uid == '0db67919c0d1673c02769194d631ad9a']

,year,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,annual_salary,...,hire_month,hire_day,resign_year,resign_month,resign_day,agency,uid,pay_effective_year,rank_year,dept_year
107,2018,ruble,charles,p,5020,police-uniform patrol bureau,5005,police officer,active,45210.88,...,10,19,,,,Baton Rouge CSD,0db67919c0d1673c02769194d631ad9a,NaN,NaN,NaN
1063,2019,ruble,charles,p,5070,police-criminal investigation,5005,police officer,active,46396.00,...,10,19,,,,Baton Rouge CSD,0db67919c0d1673c02769194d631ad9a,2019.0,NaN,2019.0
